### 1. Generating Data

In [1]:
from ecog2vec.data_generator import NeuralDataGenerator

In [4]:
efc401 = NeuralDataGenerator('/NWB/EFC401', 'EFC401')
efc401.bad_electrodes = [1,2,63,64,65,127,143,193,194,195,196,235,239,243,252,254,255,256]
efc401.bad_electrodes = [x - 1 for x in efc401.bad_electrodes]
efc401.good_electrodes = [x for x in efc401.good_electrodes if x not in efc401.bad_electrodes]

# import random

# efc400 = NeuralDataGenerator('/NWB/EFC400', 'EFC400')
# efc400.bad_electrodes = [1, 2, 33, 50, 54, 64, 128, 129, 193, 194, 256]

# available_numbers = [num for num in range(1, 257) if num not in efc400.bad_electrodes]
# random_numbers = random.sample(available_numbers, 7)
# efc400.bad_electrodes = efc400.bad_electrodes + random_numbers

# efc400.bad_electrodes = [x - 1 for x in efc400.bad_electrodes]
# efc400.good_electrodes = [x for x in efc400.good_electrodes if x not in efc400.bad_electrodes]

In [5]:
import os
# Save clipped data for training,
# save unclipped data to extract features from,
# save entire recording as chunks for training,
# save entire reocording to extract features from
parent_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs'

chopped_sentence_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/chopped_sentence'
sentence_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/sentence'
chopped_recording_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/chopped_recording'
full_recording_dir = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/full_recording'

for directory in [parent_dir, chopped_sentence_dir, sentence_dir, chopped_recording_dir, full_recording_dir]:
    os.makedirs(directory, exist_ok=True)
    print(f"Directory created or already exists: {directory}")

chunk_length = 200000

block_list = []

# 'write_raw_data', but really it's filtered to the high
# gamma range and the analytic amplitude is calculated
efc401.write_raw_data(
    # chopped_sentence_dir=chopped_sentence_dir,
    # sentence_dir=sentence_dir,
    full_recording_dir=full_recording_dir,
    chopped_recording_dir=chopped_recording_dir,
    chunk_length=chunk_length)

Directory created or already exists: /home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs
Directory created or already exists: /home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/chopped_sentence
Directory created or already exists: /home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/sentence
Directory created or already exists: /home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/chopped_recording
Directory created or already exists: /home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/full_recording


/home/bayuan/anaconda3/envs/ecog2txt/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.6.0 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/bayuan/anaconda3/envs/ecog2txt/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.3.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/bayuan/anaconda3/envs/ecog2txt/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.6.0 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/home/bayuan/anaconda3/envs/ecog2txt/lib/python3.9/site-packages/hdmf/spec/namespace.py:531: UserWarning: Ignoring cached na

### 2. Train and extract `wav2vec` features

Split into test/valid and train.

Strides should downsample by ~30x; `wav2vec` takes 16kHz inputs and our raw data is at 3kHz

In [4]:
# Borrowed from facebookresearch/fairseq with minor modifications

!python3 /home/bayuan/Documents/fall23/fairseq/examples/wav2vec/wav2vec_manifest.py \
    /home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/chopped_recording \
  --dest /home/bayuan/Documents/fall23/ecog2vec/manifest \
  --ext wav \
  --valid-percent 0.1 \
  --seed 42

In [10]:
!fairseq-hydra-train \
    task.data=/home/bayuan/Documents/fall23/ecog2vec/manifest \
    --config-dir /home/bayuan/Documents/fall23/fairseq/examples/wav2vec/config/pretraining \
    --config-name wav2vec2_base_librispeech

2023-12-19 10:43:06.186514: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-19 10:43:06.186554: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-19 10:43:06.187772: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-19 10:43:06.192802: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-19 10:43:06.718641: W tensorflow/compiler/tf2

extract $c$ embeddings

extract entire recording, then chop up into sentences


In [1]:
# Borrowed from facebookresearch/fairseq with minor modifications

import torch
import fairseq
# from scipy.io import wavfile
import os
import soundfile as sf
import numpy as np

cp_path = '/home/bayuan/Documents/fall23/ecog2vec/notebooks/outputs/2023-12-20/15-32-57/checkpoints/checkpoint_best.pt'#'/path/to/wav2vec.pt'
model, cfg, task = fairseq.checkpoint_utils.load_model_ensemble_and_task([cp_path])
model = model[0]
model.eval()

dir_path = '/home/bayuan/Documents/fall23/ecog2vec/wav2vec_inputs/full_recording'

for file in os.listdir(dir_path):

    wav_path = str(os.path.join(dir_path, file))

    wav_input_16khz, sr = sf.read(wav_path)
    print(type(wav_input_16khz))
    # 
    wav_input_16khz = wav_input_16khz.T
    wav_input_16khz = wav_input_16khz.reshape(1, 238, -1)#wav_input_16khz[np.newaxis, :, :] # change to 256; 480

    wav_input_16khz = torch.from_numpy(wav_input_16khz).to(torch.float)
    # print(wav_input_16khz.shape)

    # print(sr, wav_input_16khz.shape)
    z = model.feature_extractor(wav_input_16khz)
    c = model.feature_aggregator(z)
    
    torch.save(c, f"/home/bayuan/Documents/fall23/ecog2vec/wav2vec_outputs/latent_sentence/{file}.pt")

2023-12-24 09:26:57.079053: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-24 09:26:57.079088: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-24 09:26:57.080273: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-24 09:26:57.085988: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-24 09:26:57.633080: W tensorflow/compiler/tf2

<class 'numpy.ndarray'>


AttributeError: 'Wav2Vec2Model' object has no attribute 'feature_aggregator'